In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta

In [4]:
# Global immutable statics, used as labels
BUGS_ON_COMPLETED_STORIES = 'BUGS_ON_COMPLETED_STORIES'
BUGS_ON_UNFINISHED_STORIES = 'BUGS_ON_UNFINISHED_STORIES'
UNFINISHED_STORIES = 'UNFINISHED_STORIES'
UNPLANNED = 'UNPLANNED'
OWNER_TBD = 'OWNER_TBD'
CURRENT_SPRINT = 'CURRENT_SPRINT'
NEXT_SPRINT = 'NEXT_SPRINT'

In [5]:
# Global variable used to have a single counter for user stories id as they get generated in multiple calls. 
# Mutable state
NEXT_USER_STORY_ID = 1

<h1>Data Structures</h1>

In [6]:
# Represents a unit of *planned* work, intended to be completed in one sprint by 1 developer based on a specification from
# a product manager
#
# -userStoryId: a string unique identifier of this user story. Example: "UserStory #65"
# -originalEstimate: a float, representing the number of man-days user story was supposed to take. Estimated by developer 
# based on the spec.
# -developer: name of the developer responsible for this user story.
# -productManager: name of product manager who wrote the original spec for this user story
#
class UserStory:
    def __init__(self, userStoryId, originalEstimate, developer, productManager):
        self.userStoryId = userStoryId
        self.originalEstimate = originalEstimate
        self.developer = developer
        self.productManager = productManager

In [7]:
# Represents an in-memory proxy for a repo of user stories.
#
# -stories: a list of UserStory objects.
class UserStoriesRepo:
    def __init__(self, stories):
        self.stories = stories
        
    def findStory(self, storyId):
        for story in self.stories:
            if story.userStoryId == storyId:
                return story
        return None

In [8]:
# Represents a *non-planned* task needed to complete a user story. Typically these might be: bugs, or the need to rework
# something because the spec was misunderstood by the developer.
#
# -ticketId: integer unique identifier for this ticket.
# -userStoryId: a string unique identifier of the user story for which this is an task. Example: "UserStory #65"
# -costToFix: a non-negative integer to convey how difficulty it will be to fix. Recommended values between 1 and 10.
# -effortToDate: number of man-days spent fixing this ticket so far
# -percentAchieved: percentage of issue that has been fixed
class Ticket:
    def __init__(self, ticketId, userStoryId, costToFix):
        self.ticketId = ticketId
        self.userStoryId = userStoryId
        self.costToFix = costToFix
        self.effortToDate = 0.0
        self.percentAchieved = 0.0
     
    # Computes how long (in man-days) it should take to complete this ticket.
    def estimateRemainingEffort(self): 
        #TODO - implement a real algorithm based on probalistic forecast using costToFix and maybe history of attempts
        # to fix it.
        if (self.effortToDate >= 2.0): #for now hardcode to 2 days to fix the ticket
            return 0.0
        return 2.0 - self.effortToDate
    

In [9]:
# Represents an in-memory proxy for a repo of tickets
#
# -tickets: a list of Ticket objects.
class TicketsRepo:
    def __init__(self, tickets):
        self.tickets = tickets
        
    def findTicket(self, ticketId):
        for ticket in self.tickets:
            if ticket.ticketId == ticketId:
                return ticket
        return None
           

In [10]:
# Represents an item of work. This class is used to list all the possible things to do in a sprint (such as finishing
# a user story or fixing a ticket).
# 
# -userStoryId: the user story for which this is part of the work
# -taskType: string. Can be one of: DEV (to develop the user story) and REWORK (to fix tickets)
# -estimate: a float representing the amount of time the work it was believed to take at the start of the sprint, in man-days.
# -percentAchieved: a float representing the percentage of the work that got finished by the end of the sprint after spending
# the 'estimate' number of man-days.
# -owner: a string representing the person responsible for implementing this user story, as well as fixing anything wrong
# with it.
# -ticketId: identifier of the ticket for work items with a taskType of 'REWORK'. Otherwise it is not set
class WorkItem:
    def __init__(self, userStoryId, taskType, estimate, percentAchieved, owner, sprintPlanned, ticketId = None):
        self.userStoryId = userStoryId
        self.taskType = taskType
        self.ticketId = ticketId #Only pertinent if the taskType is REWORK
        self.estimate = estimate
        self.percentAchieved = percentAchieved
        self.owner = owner
        
        # Set at the end of a sprint where this WorkItem is assigned. If work is completed wno delays exist, 
        # will match the self.estimate when it is set
        self.actual = 0.0
        
        self.sprintPlanned = sprintPlanned # Records in which sprint this user story was first planned to be delivered in

        

In [11]:
# Represents is pending to take the user story to the finishing line
#
# -userStoryId: a string unique identifier of the user story for which this is a status. Example: "UserStory #43"
# -percentAchieved: a number between 0.0 and 1.0, representing the percentage of the initial specification completed as determined
# by the responsible developer
# -pendingTickets: an array Ticket objects. Represents open non-planned work against this user story
# -owner: a string, corresponding to the person responsible for implementing this user story and fixing anything wrong with it.
class UserStoryStatus:
    def __init__(self, userStoryId, percentAchieved=0.0):
        self.userStoryId = userStoryId
        self.percentAchieved = percentAchieved
        self.pendingTickets = []
        #self.owner = owner
        self.planned = False # Determines if this user story was added to some sprint to do it
        self.sprintPlanned = -1 # Records in which sprint this user story was first planned to be delivered in
     
    # Returns an array of WorkItem objects corresponding to all the work pending for this user story.
    # -storiesRepo: a UserStoriesRepo of all the user stories. Used to retrieve some information about the user story in 
    # question
    def generateWorkItems(self, storiesRepo):
        items = []
        # One item is the work in the user story itself, if it is not already implemented
        story = storiesRepo.findStory(self.userStoryId)
        owner = OWNER_TBD
        if self.planned:
            owner = story.developer
        BUG_TASK_TYPE = BUGS_ON_COMPLETED_STORIES
        if self.percentAchieved < 1.0:
            item = WorkItem(self.userStoryId, UNFINISHED_STORIES, story.originalEstimate*(1-self.percentAchieved), \
                            0.0, owner, self.sprintPlanned)
            items.append(item)
            BUG_TASK_TYPE = BUGS_ON_UNFINISHED_STORIES
        # Now add an item for each pending ticket
        for ticket in self.pendingTickets:
            item = WorkItem(self.userStoryId, BUG_TASK_TYPE, ticket.estimateRemainingEffort(), \
                            0.0, owner, self.sprintPlanned, ticket.ticketId)
            items.append(item)
        return items
    
    
    # Reduces the amount of work left based on the completed work during this sprint, but then increases
    # the pending work by adding the newTickets that have come in
    #
    # -completedWork: a list of WorkItems, corresponding to the work items that were labored on during this sprint and 
    # whose 'percentAchieved' field represents how much of the work item we managed to complete in this sprint.
    # -newTickets: a list of Tickets, representing newly issued tickets that increment our backlog
    # -ticketRepo: the unique TicketRepo for our application
    def updateStatus(self, completedWork, newTickets, ticketRepo):
        for item in completedWork:
            if (item.userStoryId != self.userStoryId):
                continue # Do nothing, as the completedWork is not for this UserStoryStatus
            if item.taskType==UNFINISHED_STORIES:
                # We developed the user story with an estimate that corresponds to a fraction of (1-percentAchieved) of the
                # user story. So if we completed only 70% of the estimate, say, then we completed 0.7*(1-percentAchieved), so we
                # still have 0.3 * (1 - percentAchieved) to finish the story. So the new value of percentAchieved is as per this
                # formula
                self.percentAchieved = 1 - (1 - item.percentAchieved)*(1-self.percentAchieved)
            else: 
                # We are working on a ticket then.
                ticket = ticketRepo.findTicket(item.ticketId)
                ticket.effortToDate = ticket.effortToDate + item.estimate
                if item.percentAchieved==1.0:
                    # we can close the ticket
                    self.pendingTickets.remove(ticket)
                else:
                    # we didn't fix the ticket fully. So update how much of it has been finished
                    ticket.percentAchieved = 1 - (1 - item.percentAchieved)*(1-ticket.percentAchieved)
            # Now we increase the back log with any new tickets
            self.pendingTickets.extend(newTickets)
                    

In [12]:
# Test the UserStoryStatus functions
def test_uss():
    repo = UserStoriesRepo([UserStory('Story A', 25, 'Joe Developer', 'Amy PM'), \
                          UserStory('Story B', 17, 'Alex Developer', 'Kate PM')])
    uss = UserStoryStatus('Story B', 0.0)
    uss.sprintPlanned = 1
    print('USS:', uss.userStoryId, 'achieved=' + str(uss.percentAchieved), 'tickets=' + str(uss.pendingTickets) \
          + ', sprintPlanned=' + str(uss.sprintPlanned))
    item = uss.generateWorkItems(repo)[0]
    print('Item#1 at start of sprint 1: ', item.userStoryId, item.taskType, item.ticketId, item.estimate, \
          ',percentAchieved=',item.percentAchieved, ',sprintPlanned=', item.sprintPlanned)
    item.percentAchieved = 0.7
    newTickets = [Ticket('Bug 100','Story B', 4), Ticket('Bug 101','Story B', 1.5)]
    bugRepo = TicketsRepo(newTickets)
    print('Item#1 at end of sprint 1: ', item.userStoryId, item.taskType, item.ticketId, ',Estimate=',item.estimate, \
          ',percentAchieved=',item.percentAchieved, ',sprintPlanned=', item.sprintPlanned)
    uss.updateStatus([item], newTickets, bugRepo)
    uss.sprintPlanned = 2
    print('USS:', uss.userStoryId, 'achieved=' + str(uss.percentAchieved), 'tickets=' + str(uss.pendingTickets) \
         + ', sprintPlanned=' + str(uss.sprintPlanned))
    items = uss.generateWorkItems(repo)
    item=items[0]
    print('Item#1 at start of sprint 2: ',item.userStoryId, item.taskType, item.ticketId, ',Estimate=',item.estimate, \
          ',percentAchieved=',item.percentAchieved, ',sprintPlanned=', item.sprintPlanned)
    item=items[1]
    print('Item#2 at start of sprint 2: ', item.userStoryId, item.taskType, item.ticketId, ',Estimate=',item.estimate, \
          ',percentAchieved=',item.percentAchieved, ',sprintPlanned=', item.sprintPlanned)
    item=items[2]
    print('Item#3 at start of sprint 2: ', item.userStoryId, item.taskType, item.ticketId, ',Estimate=',item.estimate, \
          ',percentAchieved=',item.percentAchieved, ',sprintPlanned=', item.sprintPlanned)
    items[0].percentAchieved = 0.9
    items[1].percentAchieved = 1.0
    items[2].percentAchieved = 0.5
    item=items[0]
    print('Item#1 at start of sprint 2: ',item.userStoryId, item.taskType, item.ticketId, ',Estimate=',item.estimate, \
          ',percentAchieved=',item.percentAchieved, ',sprintPlanned=', item.sprintPlanned)
    item=items[1]
    print('Item#2 at start of sprint 2: ', item.userStoryId, item.taskType, item.ticketId, ',Estimate=',item.estimate, \
          ',percentAchieved=',item.percentAchieved, ',sprintPlanned=', item.sprintPlanned)
    item=items[2]
    print('Item#3 at start of sprint 2: ', item.userStoryId, item.taskType, item.ticketId, ',Estimate=',item.estimate, \
          ',percentAchieved=',item.percentAchieved, ',sprintPlanned=', item.sprintPlanned)
    uss.updateStatus(items, [], bugRepo)
    print('USS:', uss.userStoryId, 'achieved=' + str(uss.percentAchieved), 'tickets=' + str(uss.pendingTickets))

#test_uss()

<b>EXPECTED OUTPUT OF TEST</b>
<li>USS: Story B achieved=0.0 tickets=[]
<li>Item#1 at start of sprint 1:  Story B DEV None 17.0 0.0
<li>Item#1 at end of sprint 1:  Story B DEV None 17.0 0.7
<li>USS: Story B achieved=0.7 tickets=[{__main__.Ticket object at 0x0000021F50FAE8D0}, {__main__.Ticket object at 0x0000021F50FAE908}]
<li>Item#1 at start of sprint 2:  Story B DEV None 5.1000000000000005 0.0
<li>Item#2 at start of sprint 2:  Story B REWORK Bug 100 2.0 0.0
<li>Item#3 at start of sprint 2:  Story B REWORK Bug 101 2.0 0.0
<li>Item#1 at end of sprint 2:  Story B DEV None 5.1000000000000005 0.9
<li>Item#2 at end of sprint 2:  Story B REWORK Bug 100 2.0 1.0
<li>Item#3 at end of sprint 2:  Story B REWORK Bug 101 2.0 0.5
<li>USS: Story B achieved=0.97 tickets=[{__main__.Ticket object at 0x0000021F50FAE908}] 

In [13]:
# Represents the remaining work to be done by a scrum team to complete their tasks for a release.
#
# -TeamId: string identifiying the scrum team for which this is a backlog. For example: "Scrum Team B"
# -pendingUserStories: array of UserStoryStatus objects. It is the universe of all that a team has yet to do. Initially
# it is an empty array but as a GA Cycle is launced, the team should get assigned all user stories pertinent to their
# area and the initial UserStoryStatus would probably be 0% done, 0 tickets, and then evolve from that from one sprint 
# to the other.
class Backlog:
    def __init__(self, teamId):
        self.pendingUserStories = []
    
    # Returns an array of WorkItem objects corresponding to all the work implied by this backlog
    # -repo: a UserStoriesRepo of all the user stories. Used to retrieve some information about the user story in 
    # question
    def generateWorkItems(self, storiesRepo):
        items = []
        for uss in self.pendingUserStories:
            items.extend(uss.generateWorkItems(storiesRepo))
        return items
    
    # Reduces the amount of work left based on the completed work during this sprint, but then increases
    # the pending work by adding the newTickets that have come in
    #
    # -completedWork: a list of WorkItems, corresponding to the work items that were labored on during this sprint and 
    # whose 'percentAchieved' field represents how much of the work item we managed to complete in this sprint.
    # -newTickets: a list of Tickets, representing newly issued tickets that increment our backlog
    # -ticketRepo: the unique TicketRepo for our application
    def updateStatus(self, completedWork, newTickets, ticketRepo):
        for uss in self.pendingUserStories:
            uss.updateStatus(completedWork, newTickets, ticketRepo)
            
    # Looks into self.pendingUserStories and returns the SUserstoryStatus instance that matches the given storyId, or
    # None if there is no match
    def getUserStoryStatus(self, storyId):
        for uss in self.pendingUserStories:
            if uss.userStoryId == storyId:
                return uss
        return None
    

In [14]:
# Test the backlog

#TODO

In [15]:
# Represents the composition and duties of a scrum team
#
# -TeamId: string identifiying this scrum team. For example: "Scrum Team B"
# -developers: array of strings. Represents the developers (including QA) in this team. Each array element is the 
# name of a developer.
# -productManagers: array of strings. Represents the productManager(s) responsible for the specs that this scrumTeam must
# implement.
# -areasOfResonsibility: array of strings. Represents the functional areas that this scrum team has responsibility for.
# For example: '[Doctor, Patient]' for a Theia scrum team that is responsible for both the 'Doctor' and 'Patient' bounded
#contexts.
# -backLog: a BackLog object. Represents all the remaning work that this scrum team has to do for the release.
class ScrumTeam:
    def __init__(self, teamId, developers=[], productManagers=[], areasOfResponsibility=[]):
        self.teamId = teamId
        self.developers = developers
        self.productManagers = productManagers
        self.areasOfResponsibility = areasOfResponsibility
        self.backlog = Backlog(teamId)

In [16]:
# Represents an in-memory proxy for a repo of scrum teams.
#
# -stories: a list of UserStory objects.
class ScrumTeamsRepo:
    def __init__(self, teams):
        self.teams = teams
        
    def findTeam(self, teamId):
        for team in self.teams:
            if team.teamId == teamId:
                return team
        return None
    
    def getUserStoryStatus(self, storyId):
        for team in self.teams:
            for uss in team.backlog.pendingUserStories:
                if uss.userStoryId == storyId:
                    return uss
        return None

<h1>People</h1>

In [17]:
# Create the teams, as a DataFrame.
# Each team is a ScrumTeam object, but we also unwrap (i.e., duplicate) a team's attributes into columns 
# of the dataframe created so that by viewing the dataframe we can verify that ScrumTeam objects were created correctly. 
# The 'Scrum Team' column in the dataframe is the actual ScrumTeam object, containing in object form what the other 
# dataframe columns display
#
# -developers_df: dataFrame of the developer information, each row having a developers name, scrum team label, and areas
# that developer works on (i.e., which bounded contexts)
# -productManagers_df: dataframe of the productManagers information, each row having the PM name and the areas for which
# that PM writes specs.
#
def createTeamsDF(developers_df, productManagers_df):
    teams_dict = {'Team Id': [], 'Developers': [], 'Product Managers': [], 'Areas of Responsibility': [], 'Scrum Team': []}
    teamLabels = dev_df['Scrum Team'].unique()
    for l in teamLabels:
        teamId = 'Team ' + l
        developers = list(dev_df[dev_df['Scrum Team'] == l]['Name'].apply(lambda x: x.rstrip('\xa0').strip()))
        areasOfResponsibility = list(dev_df[dev_df['Scrum Team'] == l]['Bounded Context'].unique())
        productManagers = []
        for area in areasOfResponsibility:
            productManagers.extend(list(pm_df[pm_df['Bounded Context']==area]['PM'].apply(lambda x: x.rstrip('\xa0').strip())))
        team = ScrumTeam(teamId, developers, productManagers, areasOfResponsibility)
        teams_dict['Team Id'].append(team.teamId)
        teams_dict['Developers'].append(team.developers)
        teams_dict['Product Managers'].append(team.productManagers)
        teams_dict['Areas of Responsibility'].append(team.areasOfResponsibility)    
        teams_dict['Scrum Team'].append(team) # this duplicates the other entries, but is to have information also packaged as an object
    teams_df = pd.DataFrame(teams_dict)
    return teams_df

<h1>User Stories</h1>
<p>Capacity planning for the release: we create a backlog of stories. Initial resource allocations are made, but they might change in a sprint if needed</p>

In [18]:
# Returns a random element from an array. Returns None if array is empty
def pickOne(array):
    if len(array)==0:
        return None
    return array[np.random.randint(0, len(array))]
                 
# Returns a random index from a Pandas Series
def pickOneIdx(series):
    return series.index[np.random.randint(0, len(series))]   

# Returns an integer corresponding to a random duration between 1 day and the maxDuration. 
def pickHowLong(maxDuration):
    return np.random.randint(1, maxDuration +1)

In [19]:
# Returns a UserStory, randomly choosing the amount of effort the UserStory might take (while fitting withing a sprint),
# and assigning to a random developer in the team with enough time to do it, to a spec written by a randomly chosen 
# product manager from the team.
#
# Depletes the time this UserStory would take from the bandwidth for the developer in question.
#
# If no developer in the team has time to do such a UserStory, returns 'None'
def generateNextUserStory(nextId, team, bandwidth, sprintDuration):
    estimate = pickHowLong(sprintDuration)

    available = bandwidth[bandwidth >= estimate] # Subset of developers with enough time to do this user story
    if (len(available) == 0):
        return None
    
    developer = pickOneIdx(available)
    bandwidth[developer] -= estimate # deplete capacity now earmarked for this user story

    productManager = pickOne(team.productManagers)
    
    return UserStory('UserStory #' + str(nextId), estimate, developer, productManager)

In [20]:
# Helper function used in createUserStoryBacklog.
# Returns a boolen on whether the team has enough capacity left in 'bandwidth' to resource one more user story
#
# -bandwidth: Pandas Series indexed on developers' names, with the values being the amount of unallocated days
# for that developer.
# -sprintDuration: integer representing duration of a sprint, in number of days 
#
def canResourceMoreWork(bandwidth, sprintDuration):
    # If over half the developers in the team have at least half a sprint left, still can resource more work
    numberOfDevelopers = bandwidth.size
    condition1 = bandwidth[bandwidth > sprintDuration/2.0].size > numberOfDevelopers / 2.0
    
    # If any developer in the team has more than 150% of a sprint left, then can still resource more work
    condition2 = bandwidth.max() >= 1.5 * sprintDuration
    
    return condition1 or condition2

In [21]:


#Creates user stories that a scrum team should work on for a release. Algorithm basically determines how much time 
# there is in a planned duration for a release, and based on that randomly creates user stories associated with the
# areas of responsibility of the scrum team in question, until capacity is filled to between 90% and 100%. Each user story 
# is supposed to be completed in a sprint, so a user story's estimated duration is ramdomly generated to be between
# 20% and 100% of the sprint's duration period.
#
# -team: the ScrumTeam for which the backlog is being created
# -releaseDuration: integer number of business days to the intended release date. Defaults to around 6 months
# -sprintDuration: integer number of business days that a sprint lasts. Defaults to 2 weeks.
def createUserStoryBacklog(team, releaseDuration=125, sprintDuration=10):
    global NEXT_USER_STORY_ID
    numberOfDevelopers = len(team.developers)
    initialCapacity = numberOfDevelopers *releaseDuration # number of work-days available for the release
    bandwidth = pd.Series(releaseDuration, team.developers) # initialize capacity to all the time left in the release

    #numberOfSprints = releaseDuration/sprintDuration
    stories = []
    backlog = Backlog(team.teamId)     
    
    # We allocate as much of the team as practical, with user stories ranging in estimated duration from 1 day
    # to the number of days in a sprint. To avoid artificial situations, don't aim to fill every single day,
    # so will stop when we have allocated at least 10% and there is no developer left with enough bandwidth
    # to resource a medium-size user story.
    # To avoid infinite loops, we force a stop after 1000 tries, though most likely we will exit well before then.
    while (canResourceMoreWork(bandwidth, sprintDuration) and NEXT_USER_STORY_ID < 10000): #To avoid infinite loops, cap number of user stories
        story = generateNextUserStory(NEXT_USER_STORY_ID, team, bandwidth, sprintDuration) #this call mutates bandwidth
        if (story == None):
            continue #Try again, maybe we tried for a very large estimate but a smaller user story can still be resourced
        stories.append(story)
        
        NEXT_USER_STORY_ID += 1
        backlog.pendingUserStories.append(UserStoryStatus(story.userStoryId))       
    
    return stories, backlog

    


In [22]:
# Test the code for creating user stories by constructing them and arranging them in a dataframe to display and evidence
# visually that the code works as it should
# 
STORIES_REPO = UserStoriesRepo([])
TEAMS_REPO = ScrumTeamsRepo([])

def initTestData(developers_df, productManagers_df, releaseDuration):
    global STORIES_REPO
    global TEAMS_REPO
    global NEXT_USER_STORY_ID
    STORIES_REPO = UserStoriesRepo([])
    TEAMS_REPO = ScrumTeamsRepo([])
    TICKETS_REPO = TicketsRepo([])
    NEXT_USER_STORY_ID = 1
    #cols = ['User Story Id','Scrum Team', 'Product Manager', 'Developer', 'Estimate',]
    userStoryId_vals = []
    scrumTeam_vals = []
    developer_vals = []
    productManager_vals = []
    estimate_vals = []
    
    teams_df = createTeamsDF(developers_df, productManagers_df)
    
    for team in teams_df['Scrum Team']:
        stories, backlog = createUserStoryBacklog(team, releaseDuration)
        STORIES_REPO.stories.extend(stories)
        TEAMS_REPO.teams.append(team)
        for story in stories:
            scrumTeam_vals.append(team.teamId)
            userStoryId_vals.append(story.userStoryId)
            developer_vals.append(story.developer)
            productManager_vals.append(story.productManager)
            estimate_vals.append(story.originalEstimate)
        team.backlog = backlog
    stories_dict = {'User Story Id': userStoryId_vals, 'Scrum Team': scrumTeam_vals, 'Product Manager':productManager_vals, \
                'Developer': developer_vals, 'Estimate': estimate_vals}
    return teams_df, pd.DataFrame(stories_dict)


<h1>Do a sprint</h1>

In [23]:
# Main data structure class to maintain how work is assigned to developers of a particular Team at a particular point in time, e.g.,
# for a particular sprint. It optionally can maintain work assigned to developers for the *next* sprint, but which
# might be started during the current sprint if developers finish early their current sprint's obligations. 
#
# -teamId: identifies the team whose currently assigned work is described by this 'WorkAssignment' instance
# -teamRepo: the TeamRepo where all teams' information is stored.
# -storiesRepo: the UserStoriesRepo where all user stories are stored
# -allocations: a nested dictionary to partition the backlog into indidual buckets based on whether they are planned, 
# in which case for which sprint (CURRENT_SPRINT, NEXT_SPRINT, UNPLANNED) and the developer assigned to, or whether they are unplanned.
# "Normally", the NEXT_SPRINT bucket would be "empty", since at the start of the sprint we assign only work for the
# CURRENT_SPRINT, and the rest of the backlog should therefore be UNPLANNED. However, since UserStories are not all of the
# same size, it can happen that developers finish their CURRENT_SPRINT early, so rather than keeping idle they shoulds start
# to progress some of the work for the NEXT_SPRINT. That is why that bucket exists, to give them something to do if
# all is delivered for the CURRENT_SPRINT, even if they can only be delivered by the NEXT_SPRINT
#
# Example of the 'allocations' attribute of this class:
#
# {CURRENT_SPRINT: {'Joe Developer':      {UNFINISHED_STORIES:          [list of WorkItems ...],
#                                          BUGS_ON_UNFINISHED_STORIES:  [list of WorkItems ...],
#                                          BUGS_ON_COMPLETED_STORIES:   [list of WorkItems ...]},
#                   'Gladys Developer':   {UNFINISHED_STORIES:          [list of WorkItems ...],
#                                          BUGS_ON_UNFINISHED_STORIES:  [list of WorkItems ...],
#                                          BUGS_ON_COMPLETED_STORIES:   [list of WorkItems ...]}    },                                                                                                        
#  NEXT_SPRINT:    {'Joe Developer':      {UNFINISHED_STORIES:          [list of WorkItems ...],
#                                          BUGS_ON_UNFINISHED_STORIES:  [list of WorkItems ...],
#                                          BUGS_ON_COMPLETED_STORIES:   [list of WorkItems ...]},
#                   'Gladys Developer':   {UNFINISHED_STORIES:          [list of WorkItems ...],
#                                          BUGS_ON_UNFINISHED_STORIES:  [list of WorkItems ...],
#                                          BUGS_ON_COMPLETED_STORIES:   [list of WorkItems ...]}    },
#  UNPLANNED:      {OWNER_TBD:            {UNFINISHED_STORIES:          [list of WorkItems ...],
#                                          BUGS_ON_UNFINISHED_STORIES:  [list of WorkItems ...],
#                                          BUGS_ON_COMPLETED_STORIES:   [list of WorkItems ...]}    }    }
class WorkAssignments:
    
    def assign(self, item, owner, preferredBucket):
        # Needs to record owner in two places: in the self.allocations, and if the owner is a real person (not the
        # OWNER_TBD global, then that means the UserStoryStatus in question is officially planned, so must
        # mutate the UserStoryStatus to reflect that.
        # Potentially the owner being assigned differs from the original developer associated to the User Story
        # at the start of the release (that association was driven purely from capacity planning, not the more micro
        # sprint planning we are doing now). So if the owner is a real person then need to update the user story as well
        story = self.storiesRepo.findStory(item.userStoryId)
        team = self.teamRepo.findTeam(self.teamId)
        uss = team.backlog.getUserStoryStatus(item.userStoryId)
        
        item.owner = owner # Update who owns this item
        
        if (owner != OWNER_TBD):
            # We actually are giving this to someone, which means it is a planned sprint deliverable
            story.developer = owner # May change the developer who initially had this in the backlog, still same scrum team
            uss.planned = True
            if (uss.sprintPlanned == -1): #This is the first sprint for which we are planning to work on this story
                uss.sprintPlanned = self.sprint
                item.sprintPlanned = self.sprint #Needed as item as the uss's old value for sprintPlanned when item was created

        bucketToUse = preferredBucket
        if (owner == OWNER_TBD):
            bucketToUse = UNPLANNED
            
        if (item.taskType == BUGS_ON_COMPLETED_STORIES):
            self.allocations[bucketToUse][owner][BUGS_ON_COMPLETED_STORIES].append(item)
        else:
            if (item.taskType == BUGS_ON_UNFINISHED_STORIES):            
                self.allocations[bucketToUse][owner][BUGS_ON_UNFINISHED_STORIES].append(item)
            else:
                self.allocations[bucketToUse][owner][UNFINISHED_STORIES].append(item)
    
    def unAssign(self, item, owner, preferredBucket):
        oldOwner = item.owner
        bucketToUse = preferredBucket
        if (oldOwner == None):
            oldOwner = OWNER_TBD
        if (oldOwner == OWNER_TBD):
            bucketToUse = UNPLANNED
            
        x = self.allocations[bucketToUse][oldOwner][BUGS_ON_COMPLETED_STORIES]
        if item in x:
            x.remove(item)
        x = self.allocations[bucketToUse][oldOwner][BUGS_ON_UNFINISHED_STORIES]
        if item in x:
            x.remove(item)        
        x = self.allocations[bucketToUse][oldOwner][UNFINISHED_STORIES]
        if item in x:
            x.remove(item)

    def reAssign(self, item, owner, preferredBucket):
        self.unAssign(item, owner, preferredBucket)
        self.assign(item, owner, preferredBucket)
          
    def __init__(self, teamId, teamRepo, storiesRepo, sprint):
        self.teamId = teamId
        self.teamRepo = teamRepo
        self.storiesRepo = storiesRepo
        self.sprint = sprint
        
        # Set the initial value of 'allocations' by looking at all work items in the team's backlog, and then
        # classifying them into several buckets:
        self.allocations = {CURRENT_SPRINT: {}, NEXT_SPRINT: {}, UNPLANNED: {}}
        team = self.teamRepo.findTeam(self.teamId)
        for person in team.developers:
            for bucket in [CURRENT_SPRINT, NEXT_SPRINT]:
                self.allocations[bucket][person] = {BUGS_ON_COMPLETED_STORIES: [], \
                                                    BUGS_ON_UNFINISHED_STORIES: [], \
                                                    UNFINISHED_STORIES: []}

        self.allocations[UNPLANNED][OWNER_TBD] = {BUGS_ON_COMPLETED_STORIES: [], BUGS_ON_UNFINISHED_STORIES: [], UNFINISHED_STORIES: []}
        
        work = team.backlog.generateWorkItems(self.storiesRepo)
        for item in work:
            self.assign(item, item.owner, CURRENT_SPRINT)

    # Returns the list of WorkItem objects that are owned by the given owner
    #
    # -owner: a string, corresponding to the person whose tasks we want to get
    # -bucket: is either CURRENT_SPRINT or NEXT_SPRINT
    def getWorkItems(self, owner, bucket):
        tasks = []
        tasks.extend(self.allocations[bucket][owner][BUGS_ON_COMPLETED_STORIES])
        tasks.extend(self.allocations[bucket][owner][BUGS_ON_UNFINISHED_STORIES])
        tasks.extend(self.allocations[bucket][owner][UNFINISHED_STORIES])
        
        return tasks

    # Helper method invoked inside the method committedTime
    def populate(self, load, capacity):
        rejects_time = 0.0
        rejects_count = 0
        debug_time = 0.0
        debug_count = 0
        implementation_time = 0.0
        implementation_count = 0
        bandwidth = capacity
            
        for item in load[BUGS_ON_COMPLETED_STORIES]:
            time_delta = item.estimate * (1-item.percentAchieved)
            rejects_time += time_delta
            rejects_count += 1
            bandwidth -= time_delta
        for item in load[BUGS_ON_UNFINISHED_STORIES]:
            time_delta = item.estimate * (1-item.percentAchieved)
            debug_time += time_delta
            debug_count += 1
            bandwidth -= time_delta
        for item in load[UNFINISHED_STORIES]:
            time_delta = item.estimate * (1-item.percentAchieved)
            implementation_time += time_delta
            implementation_count += 1
            bandwidth -= time_delta
        return rejects_time, rejects_count, debug_time, debug_count, implementation_time, implementation_count, bandwidth
        
    
    # Returns a DataFrame with information of how much effort is already committed per developer to the
    # three different pairs buckets, each pair corresponding to the time (in man-days) and the count (number of
    # tickets or user stories) for these categories: 'Rejects' (BUGS_ON_COMPLETED_STORIES), 'Debugging' (BUGS_ON_UNFINISHED_STORIES),
    # and 'Implementation' (UNFINISHED_STORIES). For each there is a '(days)' suffix for man-days and a '(#)' suffix for counts
    #
    # Result includes a 'Bandwidth' column indicating what capacity is left within the given capacity (in man-days)
    def committedTime(self, capacity):

        team = self.teamRepo.findTeam(self.teamId)
        #cols = ['Developer', 'Rejects (days)', 'Rejects (#)', 'Debugging (days)', 'Debugging (#)', \
        #        'Implementation (days)', 'Implementation (#)', 'Bandwidth']
        result_dict = {'Developer': [], 'Rejects (days)': [], 'Rejects (#)': [], 'Debugging (days)': [], 'Debugging (#)': [], \
                       'Implementation (days)': [], 'Implementation (#)': [], 'Bandwidth': [], \
                       'NEXT SPRINT (days)': [], 'NEXT SPRINT (#)': [], 'NEXT SPRINT Bandwidth': []}
        owners = team.developers.copy()
        owners.append(OWNER_TBD)
        for person in owners:
            result_dict['Developer'].append(person)

            load = {}
            if (person == OWNER_TBD):
                load = self.allocations[UNPLANNED][person]
            else:
                load = self.allocations[CURRENT_SPRINT][person]

            r_d, r_c, d_d, d_c, i_d, i_c, bandwidth = self.populate(load, capacity)
        
            result_dict['Rejects (days)'].append(r_d)
            result_dict['Rejects (#)'].append(r_c)
            result_dict['Debugging (days)'].append(d_d)
            result_dict['Debugging (#)'].append(d_c)
            result_dict['Implementation (days)'].append(i_d)
            result_dict['Implementation (#)'].append(i_c)
            if (person != OWNER_TBD):
                result_dict['Bandwidth'].append(bandwidth) # What is left from this sprint, not counting next sprint acceleration
            else:
                result_dict['Bandwidth'].append(None)
            
            # Now look at what is in store for the next sprint, if anything. We start off with a capacity for the
            # next sprint increased by any leftover bandwidth from the currrent sprint.
            if (person != OWNER_TBD):
                load = self.allocations[NEXT_SPRINT][person]
                r_d, r_c, d_d, d_c, i_d, i_c, bandwidth = self.populate(load, bandwidth + capacity)
                result_dict['NEXT SPRINT (days)'].append(r_d + d_d + i_d)
                result_dict['NEXT SPRINT (#)'].append(r_c + d_c + i_c) 
                result_dict['NEXT SPRINT Bandwidth'].append(bandwidth) 
            else: # just pad with 0's the data for the NEXT SPRINT
                result_dict['NEXT SPRINT (days)'].append(0)
                result_dict['NEXT SPRINT (#)'].append(0)        
                result_dict['NEXT SPRINT Bandwidth'].append(0) 

            
                
        return pd.DataFrame(result_dict)

    # Helper method used in the implementation of method 'committedTasks'
    def committedTasks_helper(self, bucket, person, item, result_dict):
        load = self.allocations[bucket][person]
            
        for item in load[BUGS_ON_COMPLETED_STORIES]:
            result_dict['Owner'].append(person)
            result_dict['Task Type'].append(BUGS_ON_COMPLETED_STORIES)
            result_dict['User Story Id'].append(item.userStoryId)
            result_dict['Planned for Sprint'].append(item.sprintPlanned)
            result_dict['Original Estimate'].append(item.estimate)
            result_dict['Bucket'].append(bucket)
            result_dict['Effort Spent'].append(item.actual)
            result_dict['Effort Remaining'].append(item.estimate * (1-item.percentAchieved))
            result_dict['Percent Achieved'].append(item.percentAchieved)
        for item in load[BUGS_ON_UNFINISHED_STORIES]:
            result_dict['Owner'].append(person)
            result_dict['Task Type'].append(BUGS_ON_UNFINISHED_STORIES)
            result_dict['User Story Id'].append(item.userStoryId)
            result_dict['Planned for Sprint'].append(item.sprintPlanned)
            result_dict['Original Estimate'].append(item.estimate)
            result_dict['Bucket'].append(bucket)
            result_dict['Effort Spent'].append(item.actual)
            result_dict['Effort Remaining'].append(item.estimate * (1-item.percentAchieved))
            result_dict['Percent Achieved'].append(item.percentAchieved)
        for item in load[UNFINISHED_STORIES]:
            result_dict['Owner'].append(person)
            result_dict['Task Type'].append(UNFINISHED_STORIES)
            result_dict['User Story Id'].append(item.userStoryId)
            result_dict['Planned for Sprint'].append(item.sprintPlanned)
            result_dict['Original Estimate'].append(item.estimate)
            result_dict['Bucket'].append(bucket)
            result_dict['Effort Spent'].append(item.actual)
            result_dict['Effort Remaining'].append(item.estimate * (1-item.percentAchieved))
            result_dict['Percent Achieved'].append(item.percentAchieved)
    
    # Returns an informative dataframe of what user stories and tickets are allocated in this 'WorkAssignment' instance
    def committedTasks(self):       
        #cols = ['Owner', 'Task Type', 'User Story Id', 'Original Estimate', \
        #        'Effort Spent', 'Effort Remaining', 'Percent Achieved']
        result_dict = {'Owner': [], 'Task Type': [], 'User Story Id': [], 'Planned for Sprint': [], 'Original Estimate': [], \
                       'Bucket': [], 'Effort Spent': [], 'Effort Remaining': [], 'Percent Achieved': []}
        team = self.teamRepo.findTeam(self.teamId)
        for person in team.developers:
            self.committedTasks_helper(CURRENT_SPRINT, person, item, result_dict)
            self.committedTasks_helper(NEXT_SPRINT, person, item, result_dict)
            
        return pd.DataFrame(result_dict) #, columns=cols) #, index=team.developers)

In [24]:
# At the start of a sprint, elects what items a team will do in a sprint
#
# -sprintDuration: length of the sprint for which work needs to be chosen, in man-days
def chooseWhatToDoInSprint(teamId, teamsRepo, storiesRepo, sprintDuration, sprint):
    work = WorkAssignments(teamId, teamsRepo, storiesRepo, sprint)
    
    unplanned = work.allocations[UNPLANNED][OWNER_TBD]
    workToPick = []
    workToPick.extend(unplanned[BUGS_ON_COMPLETED_STORIES])
    workToPick.extend(unplanned[BUGS_ON_UNFINISHED_STORIES])
    workToPick.extend(unplanned[UNFINISHED_STORIES])   
    
    available = work.committedTime(sprintDuration)[['Developer', 'Bandwidth']]
  
    while len(workToPick) > 0:
        item = pickOne(workToPick)
        workToPick.remove(item) # In next cycle of loop don't want to encounter this item, as it would be processed by then
        timeRequired = item.estimate * (1-item.percentAchieved)
        peopleWithTimeToDoIt = list(available[available['Bandwidth'] >= timeRequired]['Developer'])
        potentialOwner = pickOne(peopleWithTimeToDoIt)
        if potentialOwner == None:
            # This WorkItem can't be done in this sprint, as nobody has enough time for the effort it requires.
            # Try with some other work item
             continue
        work.reAssign(item, potentialOwner, CURRENT_SPRINT)
        
        # Refresh who is available and for how long, given the assignment just now, for next cycle in the loop
        available = work.committedTime(sprintDuration)[['Developer', 'Bandwidth']]
        
    # Now try again, but this time allocating any unused time to deliverables for the next sprint, i.e., use
    # time left over from the current sprint to get a heat start on the work for the next sprint, borrowing next sprint's
    # capacity since we only need to deliver then

    unplanned = work.allocations[UNPLANNED][OWNER_TBD] #Update unplanned, workToPick since we changed it in prior loop
    workToPick = []
    workToPick.extend(unplanned[BUGS_ON_COMPLETED_STORIES])
    workToPick.extend(unplanned[BUGS_ON_UNFINISHED_STORIES])
    workToPick.extend(unplanned[UNFINISHED_STORIES]) 
    
    available = work.committedTime(sprintDuration)[['Developer', 'NEXT SPRINT Bandwidth']]

    while len(workToPick) > 0:
        item = pickOne(workToPick)
        workToPick.remove(item) # In next cycle of loop don't want to encounter this item, as it would be processed by then
        timeRequired = item.estimate * (1-item.percentAchieved)
        
        # Filter to only developers who have 'carry over' bandwidth from this spring into the next one
        haveCarryOver = available[available['NEXT SPRINT Bandwidth'] > sprintDuration] 
        peopleWithTimeToDoIt = list(haveCarryOver[haveCarryOver['NEXT SPRINT Bandwidth'] >= timeRequired]['Developer'])
        potentialOwner = pickOne(peopleWithTimeToDoIt)
        if potentialOwner == None:
            # This WorkItem can't be done in this sprint, as nobody has enough time for the effort it requires.
            # Try with some other work item
             continue
        work.reAssign(item, potentialOwner, NEXT_SPRINT)
        
        # Refresh who is available and for how long, given the assignment just now, for next cycle in the loop
        available = work.committedTime(sprintDuration)[['Developer', 'NEXT SPRINT Bandwidth']]
    
    
    
    return work
           

In [25]:
# Global
TICKETS_REPO = TicketsRepo([])

# Returns an array of newly generated tickets against the work done by the team
def inflowOfTickets(teamId, teamsRepo, ticketsRepo, storiesRepo):
    # TODO - NOT YET IMPLEMENTED
    return []

In [26]:
# Root class for models that simulate how actual costs differ from estimates. Usually there might be different
# implementations, each trying to capture a different dynamic with a different driver. 
class CostDriverModel:
    # TODO - default dummy implementation for now: everything took 25% time than estimated, working task by task and 
    # leaving some incomplete, so only managed to finish 80% of the work
    #
    def __init__(self):
        return
    
    # TODO - dummy implementation for now
    def runModel(self, item):
        DELAY = 0.25    
        return 1 + DELAY

In [27]:
# Computes the real cost of delivering a work item, based on a number of 'factors', which are functions implementing a model
# for what drives costs to differ from estimates.
def computeRealCost(item, models):
    cost = item.estimate;
    for m in models:
        cost *= m.runModel(item)
    return cost

In [28]:
# Helper class, used as part of the book keeping involved in delivering a sprint.
#
# Returns what portion of the budget, if any, was left over after booking the work done by the developer
#
# -items: a list of WorkItem objects that record work that should be done in a sprint by a single developer. 
# It will be mutated by this fundtion by recording what were the actual number of man-days spent on each WorkItem, and to what degree it was completed
# -budget: number of man-days that the developer was given to try to complete the WorkItems
def bookDeveloperEffort(items, budget, models):
    for task in items:
        if budget <= 0:
            break # No more tasks progressed during this sprint
        realCost = computeRealCost(task, models)
        if realCost == 0:
            continue # Boundary base. Shouldn't happen, but if someone entered an estimate of 0 don't want to divide by 0
        if realCost <= budget:
            task.percentAchieved = 1
            task.actual = realCost
            budget -= realCost
        else:
            # Can only complete part of the item
            task.percentAchieved = budget/realCost
            task.actual = budget
            budget = 0
    return budget

In [29]:
# Based on the team's productivy during the sprint, record how much of the work initially planned for the sprint
# actually gets done
#
# -work: a WorkAssignment as it was at the start of the sprint. It is mutated by this method by recording, for each
# WorkItem in the WorkAssignment, what percentage of it got accomplished
# -sprintDuration: number of man-days that sprint lasted
# -models: a list of CostDriverModel objects modeling how real costs differ from estimates
def deliverSprint(teamId, teamsRepo, ticketsRepo, storiesRepo, work, sprintDuration, models=[CostDriverModel()]): 

    team = teamsRepo.findTeam(teamId)
    for person in team.developers:
        budget = sprintDuration # Developer has up to these many days to complete work in the sprint
        
        # First, deliver the work from the current sprint
        budget = bookDeveloperEffort(work.getWorkItems(person, CURRENT_SPRINT), budget, models)
                
        # Second, developer still has some time in his/her budget for this sprint, then he/she might have started to work
        # on things for the next sprint, in which case record that progress
        budget = bookDeveloperEffort(work.getWorkItems(person, NEXT_SPRINT), budget, models)
    

In [30]:
# At the end of a sprint, updates the backlog of a team based on the work completed in this sprint
#
# -work: a WorkAssignment, which reflects what was accomplished during the sprint through the field percentAchieved
# in each of the WorkItems in the various arrays within the work.allocations dictionary
def updateBacklogAfterSprint(teamId, teamsRepo, ticketsRepo, storiesRepo, work, newTickets):
    team = teamsRepo.findTeam(teamId)
    completedWork = []
    for person in team.developers:
        completedWork.extend(work.getWorkItems(person, CURRENT_SPRINT))
        completedWork.extend(work.getWorkItems(person, NEXT_SPRINT))
    
    team.backlog.updateStatus(completedWork, newTickets, ticketsRepo) 

In [31]:
# Now test many sprints into the future, to see if eventually people have extra time and start using that extra time
# in the current sprint to get a head start on tasks for the next sprint
#
def testMultipleSprints(numberOfSprints, teamId, teamsRepo, ticketsRepo, storiesRepo, sprintDuration):
    work = None
    for i in range(numberOfSprints):
        work = chooseWhatToDoInSprint(teamId, teamsRepo, storiesRepo, sprintDuration, sprint=i+1)
        if (i== numberOfSprints -1):
            break
        deliverSprint(teamId, teamsRepo, ticketsRepo, storiesRepo, work, sprintDuration) # This mutates 'work'
        inflow = inflowOfTickets(teamId, teamsRepo, ticketsRepo, storiesRepo)
        updateBacklogAfterSprint(teamId, teamsRepo, ticketsRepo, storiesRepo, work, inflow) # Does not mutate 'work'
    return work

#teams_df, stories_df = initTestData(dev_df, pm_df, 125)
#last = testMultipleSprints(11, teamId0, TEAMS_REPO, TICKETS_REPO, STORIES_REPO, 10)

<h1>Functions to Generate Timecards</h1>

In [1]:
# Returns a datetime.date object which is several business days after the input 'start'
def addBusinessDays(start, duration):
    remainsToAdd = duration
    current = start
    while remainsToAdd > 0:
        current = current+ timedelta(1)
        weekday = current.weekday()
        if weekday >= 5: # sunday = 6
            continue
        remainsToAdd -= 1
    return current

# Test
#print('10 business days from today:')
#print(addBusinessDays(datetime(2019,2,12), 10))

In [33]:
# Runs the dynamics of release cycle, iterating through all the sprints. For each sprint a WorkAssignment work sheet is created
# for each team, recording what the team aims to do during the sprint. At the end of the sprint the work sheet is updated
# with what was actually accomplished.
#
# This function returns two dataframes: a detailed timecards dataframe (one row for each row in any of the sheets, across all 
# teams and all sprints), and a more aggregated dataframe recording the work sheet object (a WorkAssignment) for each sprint
# and team.
# The timecards dataframes has only scalars. The work sheet dataframe has WorkAssignment objects in one column, and an
# informative list of backlog items not yet planned.
#
# Input parameters:
# -teamsRepo: a ScrumTeamsRepo containing all the scrum teams that work full time in this release. It is presumed to have
# been given a backlog of user stories to complete in this release.
# -ticketsRepo: a TicketsRepo, presumed empty at the start of the release, where tickets are stored as they are reported 
# throughout the release.
# -storiesRepo: a UserStoriesRepo containing the UserStory's that are referenced in by the teams' backlogs.
def runReleaseCycle(teamsRepo, ticketsRepo, storiesRepo, startDate, sprintDuration, numberOfSprints):
    timecards_dict = {'Sprint': [], 'Date': [], 'Team': [],'Developer': [], 'User Story': [], 'Task Type': [], 'Time Spent': []}
    
    sheet_dict = {'Sprint': [], 'Team': [], 'Work Sheet': [], 'Unplanned Backlog': []}
    sprintEndDate = startDate
    for n in range(numberOfSprints):
        sprintEndDate = addBusinessDays(sprintEndDate, sprintDuration)
        for team in teamsRepo.teams:
            teamId = team.teamId

            work = chooseWhatToDoInSprint(teamId, teamsRepo, storiesRepo, sprintDuration, sprint=n+1)
            
            
            deliverSprint(teamId, teamsRepo, ticketsRepo, storiesRepo, work, sprintDuration) # This mutates 'work'
            inflow = inflowOfTickets(teamId, teamsRepo, ticketsRepo, storiesRepo)
            updateBacklogAfterSprint(teamId, teamsRepo, ticketsRepo, storiesRepo, work, inflow) # Does not mutate 'work'

            sprintOutcome = work.committedTasks()
            for index, row in sprintOutcome.iterrows():
                timecards_dict['Sprint'].append(n+1)
                timecards_dict['Date'].append(sprintEndDate)
                timecards_dict['Developer'].append(row['Owner'])
                timecards_dict['Team'].append(teamId)
                timecards_dict['User Story'].append(row['User Story Id'])
                timecards_dict['Task Type'].append(row['Task Type'])
                timecards_dict['Time Spent'].append(row['Effort Spent']) #Even 0, record it to indicate it was in scope
            # Boundary case: if we finished all work before all sprints are through, enter 0 time spent
            if (sprintOutcome.index.size == 0):
                timecards_dict['Sprint'].append(n+1)
                timecards_dict['Date'].append(sprintEndDate)
                timecards_dict['Developer'].append(None)
                timecards_dict['Team'].append(teamId)
                timecards_dict['User Story'].append(None)
                timecards_dict['Task Type'].append(None)
                timecards_dict['Time Spent'].append(0) 
            
            sheet_dict['Sprint'].append(n+1)
            sheet_dict['Team'].append(teamId)
            sheet_dict['Work Sheet'].append(work)
            
            unplanned = []
            for uss in team.backlog.pendingUserStories:
                if not uss.planned:
                    unplanned.append(uss)
            sheet_dict['Unplanned Backlog'].append(unplanned)

    return pd.DataFrame(timecards_dict), pd.DataFrame(sheet_dict)

In [34]:
# Number of user stories worked on in a Sprint, either to implement or fix bugs on them
def countUniques(seriesGroup):
    if (seriesGroup.unique()[0] == None):
        return 0
    else:
        return seriesGroup.unique().size

def ReleaseBurnout(timecard):
    bystory = timecard.groupby('User Story')
    ends = bystory['Sprint'].max()
    starts = bystory['Sprint'].min()

    bysprint = timecard.groupby('Sprint')


    #counts = bysprint['User Story'].apply(lambda x: if x.unique()[0]!= None: len(x.unique()) else: 0) 

    counts = bysprint['User Story'].apply(lambda x: countUniques(x)) 


    efforts = bysprint['Time Spent'].sum()

    d = {'Sprint': [], 'Stories Started': [], 'Stories Progressed': [], 'Stories Completed': [], 'Effort': [],\
    'Active Developers': []}

    # Initialize
    for sprint in counts.index:
        d['Sprint'].append(sprint)
        d['Stories Progressed'].append(counts[sprint])
        d['Effort'].append(efforts[sprint])

        for s,group in bysprint:
            if s==sprint: #found it
                count = group['Developer'].unique().size
                if (count == 1): #It might be that actually no developer has work left, if the 'unique' is None
                    if (None == (group['Developer'].unique())[0]):
                        count = 0
                d['Active Developers'].append(count)

        # Allocate space, we'll come back to set a value
        d['Stories Completed'].append(0)
        d['Stories Started'].append(0) 

    for story in ends.index:
        sprint = ends[story]
        if sprint in d['Sprint']:
            idx = d['Sprint'].index(sprint)
            d['Stories Completed'][idx] += 1
        else:
            d['Sprint'].append(sprint)
            d['Stories Completed'].append(1)
            d['Stories Started'].append(0) # Allocate space, we'll come back to set a value
    for story in starts.index:
        sprint = starts[story]
        if sprint in d['Sprint']:
            idx = d['Sprint'].index(sprint)
            d['Stories Started'][idx] += 1
        else:
            d['Sprint'].append(sprint)
            d['Stories Started'].append(1)
        
        
    df = pd.DataFrame(d)
    return df.sort_values(by='Sprint')